# Feature Compilation & Engineering

<b>Features to be included:</b>

1. <s>Coordinates of HDB flats</s>
2. <s>Police Station & Number of Cases</s>
3. <s>Educational Facilities</s>
4. <s>Train Stations</s>

## Import Necessary Packages

In [ ]:
# # For running in google collab
# !pip install geopandas

In [ ]:
import json, math, os, re, requests, time

import pandas as pd
import datetime
import numpy as np
from bs4 import BeautifulSoup
import json
import requests
from datetime import datetime
from datetime import timedelta
import geopandas
import matplotlib.pyplot as plt
from sklearn.neighbors import BallTree
import matplotlib.pylab as plt

from tqdm.notebook import tqdm
from sklearn.preprocessing import MultiLabelBinarizer

pd.set_option('display.max_columns', 100)

Navigate to the working directory with your data files

In [ ]:
os.chdir(os.path.join(os.getcwd(), 'Primary Dataset'))

In [ ]:
[item for item in os.listdir() if '.csv' in item or '.xlsx' in item or '.kml' in item]

['Completion_Date_Manual_Fill.csv',
 'crime cases_dataset.csv',
 'general-information-of-schools.csv',
 'hdb_dataset.csv',
 'police centre_dataset.xlsx',
 'priSch_openingDates.csv',
 'property_price_index.csv',
 'train_stations_data.csv']

## Read Datasets

In [ ]:
# raw hdb dataset
df = pd.read_csv('hdb_dataset.csv')

# police centre location dataset
npc_df = pd.read_excel('police centre_dataset.xlsx')

# property price index dataset
ppi_df = pd.read_csv('property_price_index.csv')

# crime cases record dataset
crime_df = pd.read_csv('crime cases_dataset.csv')

# primary/secondary/jc dataset
schools_df = pd.read_csv('general-information-of-schools.csv')

# missing completion date dataset for school
school_dates = pd.read_csv('priSch_openingDates.csv')

# train stations dataset
train_station_df = pd.read_csv('train_stations_data.csv')

# missing completion date dataset
completion_date_manual = pd.read_csv('Completion_Date_Manual_Fill.csv')

c:\users\kai jing\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


For running notebook in Google Collab

In [ ]:
# from google.colab import drive

# # Setting directory for google drive
# drive.mount('/content/drive/')

In [ ]:
# path = '/content/drive/MyDrive/NUS/Year 4/BT4222 - Mining Web Data for Business Insights/Group Project/Datasets/Primary Datasets/'

# # raw hdb dataset
# df = pd.read_csv( path +'hdb_dataset.csv')

# # police centre location dataset
# npc_df = pd.read_excel( path + 'police centre_dataset.xlsx')

# # property price index dataset
# ppi_df = pd.read_csv( path + 'property_price_index.csv')

# # crime cases record dataset
# crime_df = pd.read_csv( path + 'crime cases_dataset.csv')

# # primary/secondary/jc dataset
# schools_df = pd.read_csv( path + 'general-information-of-schools.csv')

# # missing completion date dataset for school
# school_dates = pd.read_csv( path + 'priSch_openingDates.csv')

# # train stations dataset
# train_station_df = pd.read_csv( path + 'train_stations_data.csv')

# # missing completion date dataset
# completion_date_manual = pd.read_csv( path + 'Completion_Date_Manual_Fill.csv')

## Data Cleaning

In [ ]:
# Remove duplicate rows
def remove_duplicates(data):
    data.drop_duplicates(inplace=True)
    data.reset_index(drop=True, inplace=True)
    
    return data

In [ ]:
# Remove 2021 data
def remove_2021(data):
    data['Sale Date'] = pd.to_datetime(data['Sale Date'])
    
    # Only keep transactions before 2021
    data = data[data['Sale Date'].apply(lambda x: x.year) < 2021]
    data.reset_index(drop=True, inplace=True)
    
    return data

In [ ]:
def clean_address(data):
    # Remove addresses without floor/unit number (landed property)
    data = data[data.Address.str.contains('#')]
    
    return data

In [ ]:
def generate_floor(data):
    # Find the floor number of the unit
    data['Floor Number'] = data.Address.apply(lambda x: x.split('#')[1].split('-')[0])
    data['Floor Number'] = data['Floor Number'].apply(lambda x: 0 if 'B' in x else int(x))
    
    return data

In [ ]:
def clean_postal(data):
    # convert 5-digit postal codes to 6-digits by padding with 0 in front
    data['Postal Code'] = data['Postal Code'].apply(lambda x: str(x).zfill(6))
    
    return data

In [ ]:
df = remove_duplicates(df)
df = remove_2021(df)
df = clean_address(df)
df = generate_floor(df)
df = clean_postal(df)

df.head()

,Project Name,Transacted Price ($),Area (SQFT),Unit Price ($ PSF),Sale Date,Address,Type of Sale,Type of Area,Area (SQM),Unit Price ($ PSM),Nett Price($),Property Type,Number of Units,Tenure,Completion Date,Purchaser Address Indicator,Postal Code,Postal District,Postal Sector,Planning Region,Planning Area,Floor Number
0,EUPHONY GARDENS,"600,000","1,022.58",587,2010-01-01,5 JALAN MATA AYER #01-25,Resale,Strata,95,"6,316",-,Condominium,1,99 yrs from 25/02/1998,2001,HDB,759151,27,75,North Region,Mandai,1
1,CARIBBEAN AT KEPPEL BAY,"1,909,000","1,485.43","1,285",2010-01-02,42 KEPPEL BAY DRIVE #01-111,Resale,Strata,138,"13,833",-,Condominium,1,99 yrs from 16/08/1999,2004,Private,098656,4,9,Central Region,Bukit Merah,1
2,ASTOR GREEN,"890,000","1,065.64",835,2010-01-03,7 PINE GROVE #10-01,Resale,Strata,99,"8,990",-,Condominium,1,99 yrs from 18/11/1991,1995,Private,597592,21,59,Central Region,Bukit Timah,10
3,ORCHID PARK CONDOMINIUM,"700,000","1,194.8",586,2010-01-03,81 YISHUN STREET 81 #06-01,Resale,Strata,111,"6,306",-,Condominium,1,99 yrs from 01/09/1991,1994,Private,768445,27,76,North Region,Yishun,6
4,CARIBBEAN AT KEPPEL BAY,"1,700,000","1,313.21","1,295",2010-01-04,12 KEPPEL BAY DRIVE #10-13,Resale,Strata,122,"13,934",-,Condominium,1,99 yrs from 16/08/1999,2004,Private,098641,4,9,Central Region,Bukit Merah,10


## Feature Engineering

### 1. Geo-Encoding (Longitude, Latitude)

In [ ]:
# Using requests to call geographic information from OneMap API
def get_info(searchVal, returnGeom=True, getAddr=True, pageNum=1):
    if returnGeom:
        returnGeom = 'Y'
    else:
        returnGeom = 'N'
        
    if getAddr:
        getAddr = 'Y'
    else:
        getAddr = 'N'
    
    url = 'https://developers.onemap.sg/commonapi/search?searchVal={}&returnGeom={}&getAddrDetails={}&pageNum={}'
    url = url.format(searchVal, returnGeom, getAddr, pageNum)
    
    return json.loads(requests.get(url).content.decode("UTF-8"))['results']

In [ ]:
# Get (long, lat) for unique postal codes only to reduce runtime
def postal_search(iterable=[]):
    data = []

    for postal in tqdm(set(iterable)):
        response = get_info(searchVal=postal)

        if len(response) != 0:
            data.append(response[0])
        else:
            continue

    df = pd.DataFrame(data)

    return df

In [ ]:
# Use postal code information to derive (lon, lat) from OneMap API
def search_by_postal(data):
    data['Postal Code'] = data['Postal Code'].apply(str)
    
    # Use postal code to extract geographical features
    df_postal = postal_search(set(data['Postal Code'].values))
    df_postal = df_postal[df_postal['POSTAL'] != 'NIL']
    df_postal.drop(columns=['SEARCHVAL', 'LONGTITUDE', 'ADDRESS', 'X', 'Y'], inplace=True)
    
    data = pd.merge(left=data, 
                    right=df_postal, 
                    how='left', 
                    left_on='Postal Code', 
                    right_on='POSTAL', 
                    copy=True)
    
    return data

In [ ]:
# Search for remaining missing lon, lat data using address instead of postal code through OneMap API
def search_by_addr(data):
    # Use truncated address (block number and street name) to extract remaining missing (lon, lat) data
    data['address_trunc'] = data['Address'].apply(lambda x: x.split(' #')[0])
    
    address_set = set([x.split(' #')[0] for x in data[data.POSTAL.isna()]['Address'].values])
    df_address = postal_search(address_set)
    
    df_address['address_trunc'] = df_address['BLK_NO'] + ' ' + df_address['ROAD_NAME']
    df_address.drop(columns=['SEARCHVAL', 'LONGTITUDE', 'ADDRESS', 'X', 'Y'], inplace=True)
    
    # Merge additional (lon, lat) data to rows in the original dataset with missing (lon, lat) data
    df_missing = data[data.POSTAL.isna()].copy()
    
    # Remove 'BLK_NO', 'ROAD_NAME', 'BUILDING', 'POSTAL', 'LATITUDE', 'LONGITUDE' columns
    # --> These are columns with missing data to be merged back from df_address
    df_missing = pd.concat([df_missing.loc[:,:'Planning Area'], df_missing.loc[:,'address_trunc':]], axis=1)
    
    missing_merged = pd.merge(left=df_missing, 
                              right=df_address, 
                              how='left', 
                              left_on='address_trunc', 
                              right_on='address_trunc', 
                              copy=True)
    
    missing_merged = pd.concat([missing_merged, missing_merged.pop('address_trunc')], axis=1)
    
    # Concatenate merged dataframe (previously with missing lon, lat info) back with the main dataframe
    data = pd.concat([data[data.POSTAL.notna()], missing_merged], axis=0)
    data.reset_index(drop=True, inplace=True)
    
    return data

In [ ]:
df = search_by_postal(df)
df = search_by_addr(df)

df.head()

  0%|          | 0/2150 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

,Project Name,Transacted Price ($),Area (SQFT),Unit Price ($ PSF),Sale Date,Address,Type of Sale,Type of Area,Area (SQM),Unit Price ($ PSM),Nett Price($),Property Type,Number of Units,Tenure,Completion Date,Purchaser Address Indicator,Postal Code,Postal District,Postal Sector,Planning Region,Planning Area,Floor Number,BLK_NO,ROAD_NAME,BUILDING,POSTAL,LATITUDE,LONGITUDE,address_trunc
0,EUPHONY GARDENS,"600,000","1,022.58",587,2010-01-01,5 JALAN MATA AYER #01-25,Resale,Strata,95,"6,316",-,Condominium,1,99 yrs from 25/02/1998,2001,HDB,759151,27,75,North Region,Mandai,1.0,5,JALAN MATA AYER,EUPHONY GARDENS,759151,1.42462147471378,103.82489908534,5 JALAN MATA AYER
1,CARIBBEAN AT KEPPEL BAY,"1,909,000","1,485.43","1,285",2010-01-02,42 KEPPEL BAY DRIVE #01-111,Resale,Strata,138,"13,833",-,Condominium,1,99 yrs from 16/08/1999,2004,Private,098656,4,9,Central Region,Bukit Merah,1.0,42,KEPPEL BAY DRIVE,CARIBBEAN AT KEPPEL BAY,098656,1.26681341314756,103.814673696844,42 KEPPEL BAY DRIVE
2,ASTOR GREEN,"890,000","1,065.64",835,2010-01-03,7 PINE GROVE #10-01,Resale,Strata,99,"8,990",-,Condominium,1,99 yrs from 18/11/1991,1995,Private,597592,21,59,Central Region,Bukit Timah,10.0,7,PINE GROVE,ASTOR GREEN,597592,1.3184590944709,103.776637736934,7 PINE GROVE
3,ORCHID PARK CONDOMINIUM,"700,000","1,194.8",586,2010-01-03,81 YISHUN STREET 81 #06-01,Resale,Strata,111,"6,306",-,Condominium,1,99 yrs from 01/09/1991,1994,Private,768445,27,76,North Region,Yishun,6.0,81,YISHUN STREET 81,ORCHID PARK CONDOMINIUM,768445,1.41345822654195,103.835186431485,81 YISHUN STREET 81
4,CARIBBEAN AT KEPPEL BAY,"1,700,000","1,313.21","1,295",2010-01-04,12 KEPPEL BAY DRIVE #10-13,Resale,Strata,122,"13,934",-,Condominium,1,99 yrs from 16/08/1999,2004,Private,098641,4,9,Central Region,Bukit Merah,10.0,12,KEPPEL BAY DRIVE,CARIBBEAN AT KEPPEL BAY,098641,1.26656974202485,103.81658730454,12 KEPPEL BAY DRIVE


In [ ]:
# To save yourself the trouble of running the API repeatedly
dataframe = df

In [ ]:
print(df.shape, '\n')
print(df.info(), '\n')
print(df.isna().sum(), '\n')

df.describe()

(55041, 29) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55041 entries, 0 to 55040
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Project Name                 55041 non-null  object        
 1   Transacted Price ($)         55041 non-null  object        
 2   Area (SQFT)                  55041 non-null  object        
 3   Unit Price ($ PSF)           55041 non-null  object        
 4   Sale Date                    55041 non-null  datetime64[ns]
 5   Address                      55041 non-null  object        
 6   Type of Sale                 55041 non-null  object        
 7   Type of Area                 55041 non-null  object        
 8   Area (SQM)                   55041 non-null  object        
 9   Unit Price ($ PSM)           55041 non-null  object        
 10  Nett Price($)                55041 non-null  object        
 11  Property Type              

,Number of Units,Postal District,Postal Sector,Floor Number
count,55041.000000,55041.000000,55041.000000,54764.000000
mean,1.000291,16.278956,46.983594,9.936528
std,0.025573,7.118863,21.029920,8.342209
min,1.000000,1.000000,1.000000,1.000000
25%,1.000000,13.000000,34.000000,4.000000
50%,1.000000,18.000000,52.000000,8.000000
75%,1.000000,22.000000,64.000000,13.000000
max,5.000000,28.000000,82.000000,71.000000


### 2. Property Price Index

Match Quarter to Timestamp in Sale Date

In [ ]:
def get_ppi(hdb_geocoded_df, ppi_df):
    length = len(hdb_geocoded_df)
    
    # create new column PPI
    hdb_geocoded_df['PPI'] = 0.0
    
    # get month and year
    for i in range(length):
        date_object = hdb_geocoded_df['Sale Date'][i]
        #date_object = datetime.strptime(date_str, '%Y-%m-%d').date()
        month = date_object.month
        year = date_object.year
        
        # sort the quarters
        if str(year) == '2021':
            pass
        elif 1 <= month <= 3:
            key = str(year) + "Q1"
        elif 4 <= month <= 6:
            key = str(year) + "Q2"
        elif 7 <= month <= 9:
            key = str(year) + "Q3"
        else:
            key = str(year) + "Q4"
        
        # get row index of corresponding quarter in ppi_df
        idx = ppi_df.index.get_loc(ppi_df.index[ppi_df['Quarter'] == key][0])
        
        # get ppi for that particular period
        ppi = ppi_df['Property Price Index of Non-Landed Residential Properties (INDEX)'][idx]
        
        # append ppi to hdb_geocoded_df
        hdb_geocoded_df['PPI'][i] = ppi
    
    return hdb_geocoded_df

hdb_geocoded_df = get_ppi(df, ppi_df)

c:\users\kai jing\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### 3. Nearest Police Centre & Number of Cases

Get Postal Code from Location Column

In [ ]:
def get_postal(npc_df):
    length = len(npc_df)
    
    # new column to store postal code
    npc_df['Postal Code'] = ''
    
    # trim last 6 characters of location i.e. postal code
    for i in range(length):
        npc_df['Postal Code'][i] = npc_df['Location'][i][-6:]
    
    return npc_df

npc_df = get_postal(npc_df)

Get Address from Location Column

In [ ]:
def get_address(npc_df):
    length = len(npc_df)
    
    # new column to store address
    npc_df['Address'] = ''
    
    # remove last 6 characters of location i.e. postal code
    for i in range(length):
        npc_df['Address'][i] = npc_df['Location'][i][:-10]
    
    return npc_df

npc_df = get_address(npc_df)

Get coordinates from OneMap API

In [ ]:
def get_response(searchVal, returnGeom=True, getAddr=True, pageNum=1):
    if returnGeom:
        returnGeom = 'Y'
    else:
        returnGeom = 'N'
        
    if getAddr:
        getAddr = 'Y'
    else:
        getAddr = 'N'
    
    url = 'https://developers.onemap.sg/commonapi/search?searchVal={}&returnGeom={}&getAddrDetails={}&pageNum={}'
    url = url.format(searchVal, returnGeom, getAddr, pageNum)
    
    return requests.get(url).content.decode("UTF-8")

def convert_dict(postal_code, centre_name, centre_add):
    response = get_response(postal_code, getAddr=False)
    response_dict = json.loads(response)
    
    # try with police centre name if postal code does not return any long/lat
    if response_dict['found'] == 0:
        response = get_response(centre_name, getAddr=False)
        response_dict = json.loads(response)
    
    # try with centre address if postal code does not return any long/lat
    if response_dict['found'] == 0:
        response = get_response(centre_add, getAddr=False)
        response_dict = json.loads(response)
        
    return response_dict['results']

def get_coord(npc_df):
    length = len(npc_df)
    
    npc_df['Longitude'] = 0.0
    npc_df['Latitude'] = 0.0
    # npc_df['X'] = ''
    # npc_df['Y'] = ''
    
    for i in range(length):
        
        # postal code of NPC
        postal_code = npc_df['Postal Code'][i]
        
        # name of NPC
        centre_name = npc_df['Police Centre'][i]
        # address of NPC
        address = npc_df['Address'][i]
        
        # get response dictionary
        response = convert_dict(postal_code, centre_name, address)
        
        # if there exist more than 1 option, select the search value that includes NEIGHBOURHOOD POLICE CENTRE
        for j in range(len(response)):
            if 'NEIGHBOURHOOD POLICE CENTRE' in response[j]['SEARCHVAL']:
                longitude = response[j]['LONGITUDE']
                latitude = response[j]['LATITUDE']
                # x_coord = response[j]['X']
                # y_coord = response[j]['Y']
        
        # substitute column value with corresponding long, lat, x, y
        npc_df['Longitude'][i] = float(longitude)
        npc_df['Latitude'][i] = float(latitude)
        # npc_df['X'][i] = x_coord
        # npc_df['Y'][i] = y_coord
        
    return npc_df

npc_df = get_coord(npc_df)

c:\users\kai jing\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\kai jing\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Create GeoDataFrame

In [ ]:
# LONGITUDE & LATITUDE in hdb_geocoded_df are strings, convert to float
hdb_geocoded_df.LATITUDE = pd.to_numeric(hdb_geocoded_df.LATITUDE, errors='coerce')
hdb_geocoded_df.LONGITUDE = pd.to_numeric(hdb_geocoded_df.LONGITUDE, errors='coerce')

In [ ]:
def create_geo(df, long_col, lat_col):
    gdf = geopandas.GeoDataFrame(
        df, geometry=geopandas.points_from_xy(df[long_col], df[lat_col]))
    
    # remove any NA instances if applicable
    gdf = gdf.dropna(subset = [long_col, lat_col])
    
    return gdf

npc_gdf = create_geo(npc_df, 'Longitude', 'Latitude')
hdb_geocoded_gdf = create_geo(hdb_geocoded_df, 'LONGITUDE', 'LATITUDE')

In [ ]:
# Save npc gdf for interface purpose
npc_gdf.to_csv('police_centre_gdf.csv', index = False, encoding='utf-8-sig')

Get nearest police stations

In [ ]:
def get_nearest(src_points, candidates, k_neighbors=1):
    """Find nearest neighbors for all source points from a set of candidate points"""

    # Create tree from the candidate points
    tree = BallTree(candidates, leaf_size=15, metric='haversine')

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    # Get closest indices and distances (i.e. array at index 0)
    # note: for the second closest points, you would take index 1, etc.
    closest = indices[0]
    closest_dist = distances[0]

    # Return indices and distances
    return (closest, closest_dist)


def nearest_neighbor(left_gdf, right_gdf, return_dist=False):
    """
    For each point in left_gdf, find closest point in right GeoDataFrame and return them.

    NOTICE: Assumes that the input Points are in WGS84 projection (lat/lon).
    """

    left_geom_col = left_gdf.geometry.name
    right_geom_col = right_gdf.geometry.name

    # Ensure that index in right gdf is formed of sequential numbers
    right = right_gdf.copy().reset_index(drop=True)

    # Parse coordinates from points and insert them into a numpy array as RADIANS
    left_radians = np.array(left_gdf[left_geom_col].apply(lambda geom: (geom.y * np.pi / 180, geom.x * np.pi / 180)).to_list())
    right_radians = np.array(right[right_geom_col].apply(lambda geom: (geom.y * np.pi / 180, geom.x * np.pi / 180)).to_list())

    # Find the nearest points
    # -----------------------
    # closest ==> index in right_gdf that corresponds to the closest point
    # dist ==> distance between the nearest neighbors (in meters)

    closest, dist = get_nearest(src_points=left_radians, candidates=right_radians)

    # Return points from right GeoDataFrame that are closest to points in left GeoDataFrame
    closest_points = right.loc[closest]

    # Ensure that the index corresponds the one in left_gdf
    closest_points = closest_points.reset_index(drop=True)

    # Add distance if requested
    if return_dist:
        # Convert to meters from radians
        earth_radius = 6371000  # meters
        closest_points['distance'] = dist * earth_radius

    return closest_points

mapped_npc = nearest_neighbor(hdb_geocoded_gdf, npc_gdf, return_dist=True)

Merge the police centre back to the respective HDB flats in main dataframe

In [ ]:
def get_npc(spatial_df, main_df):
    
    main_df = main_df.reset_index()
    
    # rename geometry in spatial_df
    spatial_df = spatial_df.rename(columns={'geometry': 'Centre geometry'})
    
    # join police centre and geometry
    main_df = main_df.join(spatial_df[['Police Centre', 'Centre geometry']])
    
    return main_df

hdb_geocoded_gdf = get_npc(mapped_npc, hdb_geocoded_gdf)

Get Crime Records Per NPC

In [ ]:
def extract_npc(crime_df):
    length = len(crime_df)
    
    # split string by separator to get the specific npc
    for i in range(length):
        original = crime_df['npc'][i]
        crime_df['npc'][i] = original.split(' - ')[1]
    
    return crime_df

crime_df = extract_npc(crime_df)

c:\users\kai jing\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Map Crime Cases into Nested Dictionary

In [ ]:
def make_dict(crime_df):
    
    # store values in nested dictionary
    d = {}
    
    # remove invalid values e.g. '-' and 'na'
    crime_df = crime_df[(crime_df['number (cases)'] != '-') & (crime_df['number (cases)'] != 'na')]
    crime_df = crime_df.reset_index()
    
    length = len(crime_df)
    
    for i in range(length):
        
        year = crime_df['year'][i]
        npc = crime_df['npc'][i]
        cases = int(crime_df['number (cases)'][i])
        
        # year does not exists as key
        if year not in d.keys():
            d[year] = {}
            d[year][npc] = cases
        
        # year exist, but npc does not
        elif npc not in d[year].keys():
            d[year][npc] = cases
        
        # both year and npc exist, new cases
        else:
            d[year][npc] = int(d[year][npc]) + cases
    
    return d

case_dict = make_dict(crime_df)

In [ ]:
# remove 2011 - 2013 entries to obtain more recent observations
def entries_to_remove(keys_to_remove, case_dict):
    list(map(case_dict.pop, keys_to_remove))
    return case_dict

def mod_dict(case_dict):
    
    # remove keys
    case_dict = entries_to_remove([2011, 2012, 2013], case_dict)
    
    # cases from 2014 - 2018; total of 5 years worth of data to work with
    years = case_dict.keys()
    new_dict = {}
    
    # sum up total cases for each npc over 8 years
    for year in years:
        for npc in case_dict[year].keys():
            # if npc does not exists in new_dict
            if npc not in new_dict.keys():
                new_dict[npc] = case_dict[year][npc]
            
            # if npc already exists
            else:
                new_dict[npc] = new_dict[npc] + case_dict[year][npc]
    
    # get average value over 5 years (2014 - 2018)
    for station in new_dict.keys():
        new_dict[station] = new_dict[station] / 5
    
    return new_dict

average_cases = mod_dict(case_dict)

In [ ]:
# Save average cases per NPC for interface purpose
avg_cases_df = pd.DataFrame.from_dict(data=average_cases, orient = 'index').reset_index()#.to_csv('dict_file.csv')
avg_cases_df.columns = ['Police Centre', 'Average Cases Per Year']
avg_cases_df.to_csv('average_cases_by_npc.csv', index=False)

In [ ]:
def append_average(hdb_geocoded_gdf, average_dict):
    length = len(hdb_geocoded_gdf)
    
    # create copy of df
    copy_df = hdb_geocoded_gdf
    
    # new column for aggregated cases
    copy_df['Average Cases Per Year'] = 0
    
    # assign average cases to each npc
    for i in range(length):
        npc = copy_df['Police Centre'][i][1:]
        npc = npc.replace(u'\xa0', u' ')
        copy_df.at[i, 'Average Cases Per Year'] = average_dict[npc]
    
    return copy_df

hdb_geocoded_gdf = append_average(hdb_geocoded_gdf, average_cases)

### 4. Number of Schools for Each Listing

Retrieve information for primary schools only from school_dataset

In [ ]:
prisch_df = schools_df[['school_name', 'postal_code', 'mainlevel_code']].loc[schools_df['mainlevel_code'] == 'PRIMARY'].reset_index(drop=True)
prisch_df = prisch_df.rename(columns = {'school_name': 'Name', 'postal_code': 'Postal Code', 'mainlevel_code': 'School Type'})

Get latitude and longitutde for primary schools

In [ ]:
def get_coordinates(df):
    latitude = []
    longitude = []
    for i in range(len(df)):
        postal_code = str(df.iloc[i]['Postal Code'])
        if len(postal_code) == 5:
            postal_code = '0' + postal_code
        response = get_info(searchVal=postal_code, getAddr=True)
        if response:
            lat = float(response[0]['LATITUDE'])
            long = float(response[0]['LONGITUDE'])
        else:
            lat = 0
            long = 0
        latitude.append(lat)
        longitude.append(long)
    df['Longitude'] = longitude
    df['Latitude'] = latitude
    df = df[['Name', 'School Type', 'Postal Code', 'Longitude', 'Latitude']]
    df = df.reset_index(drop=True)
    return df

pri_df = get_coordinates(prisch_df)

Get opening and closing dates for primary schools 
- If the primary school has no opening date found, we assume that it was opened before 2000
- If the primary school has no closing date, the closed_date is set 5 years from current date

In [ ]:
school_dates['opening_date'] = pd.to_datetime(school_dates['opening_date']).fillna(pd.to_datetime('2000-01-02'))

In [ ]:
school_dates['closed_date'] = pd.to_datetime(school_dates['closed_date']).fillna(pd.datetime.now().date()+ timedelta(days=1825))

c:\users\kai jing\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.


Merge primary school dataframe with opening and closing dates

In [ ]:
pri_df = pri_df.merge(school_dates[['school_name', 'opening_date', 'closed_date']], left_on = 'Name', right_on = 'school_name', how = 'left')
pri_df.drop(columns = 'school_name', inplace = True)

Create geo dataframes for primary schools

In [ ]:
pri_gdf = create_geo(pri_df, 'Longitude', 'Latitude')

In [ ]:
def count_sch(src_points, candidates, rad):
    """Find schools within the stated radius"""
    # Create tree from the candidate points
    tree = BallTree(candidates, leaf_size=15, metric='haversine')

    # Returns number of schools within radius
    # Get distance of nearest school
    dist, ind = tree.query(src_points, k=1)
    dist = dist*6371000
    # Count schools within radius
    count = tree.query_radius(src_points, r = rad, count_only = True)
    return count, dist.ravel()



# Return the number of schools within the distance for each apartment wrt sale date 
def nearest_sch(property_gdf, sch_gdf, dist):
    property_copy = property_gdf.copy().reset_index(drop=True)
    sch_copy = sch_gdf.copy().reset_index(drop=True)
    earth_radius = 6371000 #meters
    radius = dist / earth_radius 

    property_geom_col = property_copy.geometry.name
    sch_geom_col = sch_copy.geometry.name

    property_copy['radians'] = property_copy[property_geom_col].apply(lambda geom: [geom.y * np.pi / 180, geom.x * np.pi / 180])
    sch_copy['radians'] = sch_copy[sch_geom_col].apply(lambda geom: [geom.y * np.pi / 180, geom.x * np.pi / 180])
    
    # Take school's opening date & closed date to be one year in advanced - forward looking 
    sch_copy['adv_open_date'] = sch_copy['opening_date'] - pd.DateOffset(years=1)
    sch_copy['adv_close_date'] = sch_copy['closed_date'] - pd.DateOffset(years=1)
    property_copy['Sale Date'] = pd.to_datetime(property_copy['Sale Date'], infer_datetime_format=False)
    
    results = property_copy.apply(lambda x: count_sch([x['radians']], np.stack(sch_copy[(sch_copy['adv_open_date']<=x['Sale Date']) & (sch_copy['adv_close_date']>=x['Sale Date'])]['radians']), radius), axis = 1)
    count, nearest_dist = zip(*results)
    count = np.concatenate(count, axis=0)
    nearest_dist = np.concatenate(nearest_dist, axis=0)
    return count, nearest_dist

Map number of primary schools within 2KM and distance of nearest primary school for each listing

In [ ]:
no_of_pri, nearest_pri_dist = nearest_sch(hdb_geocoded_gdf, pri_df, 2000)
hdb_geocoded_gdf['Number of Primary Schools'] = no_of_pri
hdb_geocoded_gdf['Nearest Primary School'] = nearest_pri_dist  

In [ ]:
# Save primary school gdf for interface purpose
pri_gdf.to_csv('primary_sch_gdf.csv', header= True)

### 5. Number of Train Stations and Lines within Radius

In [ ]:
import datetime
# Preprocessing train_stations dataframe altering all train stations categorized as 'OTHERS'
# Returns geoDataFrame of the processed dataframe
def prep_train_station_df(train_stations): 
    
    # Treating datetime columns
    train_stations['OPEN_DATE'] = pd.to_datetime(train_stations['OPEN_DATE'])
    train_stations['CLOSE_DATE'] = pd.to_datetime(train_stations['CLOSE_DATE'], format = '%Y-%m-%d')
    
    # Adding a one year buffer for train stations
    train_stations['OPEN_DATE'] = train_stations['OPEN_DATE'].apply(lambda x: datetime.datetime(
                                                                            int(x.year - 1), 
                                                                            x.month, 
                                                                            1))
    train_stations['CLOSE_DATE'] = train_stations['CLOSE_DATE'].apply(lambda x: datetime.datetime(
                                                                            int(x.year - 1), 
                                                                            x.month, 
                                                                            1))
    
    # Converting train_stations from DataFrame object in GeoDataFrame
    train_station_gdf = geopandas.GeoDataFrame(train_stations, 
                                               geometry=geopandas.points_from_xy(train_stations['Longitude'], train_stations['Latitude']),
                                               crs={"init":"EPSG:4326"})

    # Converting CRS so that we can calculate distances in meters instead of degrees
    train_station_gdf = train_station_gdf.to_crs({"init": "EPSG:3857"})
    

    return train_station_gdf


# Takes in point (Shapely point object) and distance in meters (int)
# Returns 3 values as output
# Returns: List of stations within radius, List of train lines within radius, distance to nearest station
def locate_nearest_stations(point, sale_date, distance, train_station_gdf): #note that this is the distance in meters 
  
    # Creating a deep copy of the Train Stations geodataframe 
    stations = train_station_gdf.copy()
    
    # Filtering out stations that have not yet opened
    stations = stations[stations['OPEN_DATE'] <= sale_date]
    
    # Filtering out stations that have closed
    stations = stations[stations['CLOSE_DATE'] >= sale_date]

    # Computing the distance between each of the stations and the apt
    stations['distance'] = stations['geometry'].apply(lambda x: x.distance(point))

    # Finding list of unique stations and train lines within 1km radius of the listing
    within_distance_stations = stations[stations['distance'] <= distance]['STN_NAME'].unique().tolist()
    within_distance_lines = stations[stations['distance'] <= distance]['COLOR'].unique().tolist()
    nearest_station_distance = stations['distance'].min() 

    return within_distance_stations, within_distance_lines, nearest_station_distance

# Returns a dataframe of all engineered features
def train_station_features(hdb_geocoded_gdf, train_station_gdf):
  
    # Creating a deep copy of the hdb_geocoded_gdf
    hdb_gdf = hdb_geocoded_gdf.copy()

    # Converting CRS of hdb_gdf so that we can calculate distances in meters instead of degrees
    hdb_gdf = hdb_gdf.set_crs(epsg=4326)
    hdb_gdf = hdb_gdf.to_crs({"init": "EPSG:3857"})

    # Engineered features: stations_1km, train_lines_1km, nearest_station_distance
    tqdm.pandas() #Gives an estimate for how long the program will take
    features = hdb_gdf.progress_apply(lambda x: locate_nearest_stations(x['geometry'], x['Sale Date'], 1000, train_station_gdf), axis = 1)
    hdb_gdf['stations_1km'] , hdb_gdf['train_lines_1km'], hdb_gdf['nearest_station_distance'] = zip(*features)

    # Derived features: num_stations_1km, num_lines_1km
    hdb_gdf['num_stations_1km'] = hdb_gdf['stations_1km'].apply(len)
    hdb_gdf['num_lines_1km'] = hdb_gdf['train_lines_1km'].apply(len)

    return hdb_gdf[['stations_1km', 'num_stations_1km', 'train_lines_1km', 'num_lines_1km', 'nearest_station_distance']] 


In [ ]:
import datetime
# Retrieve preprocessed geoDataFrame from raw train_station_df
train_station_gdf = prep_train_station_df(train_station_df)

# Adding features created for train station
hdb_geocoded_gdf[['stations_1km', 'num_stations_1km', 'train_lines_1km', 'num_lines_1km', 'nearest_station_distance']] = train_station_features(hdb_geocoded_gdf, train_station_gdf)
hdb_geocoded_gdf.head()

c:\users\kai jing\appdata\local\programs\python\python37\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


  0%|          | 0/54764 [00:00<?, ?it/s]

c:\users\kai jing\appdata\local\programs\python\python37\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


,index,Project Name,Transacted Price ($),Area (SQFT),Unit Price ($ PSF),Sale Date,Address,Type of Sale,Type of Area,Area (SQM),Unit Price ($ PSM),Nett Price($),Property Type,Number of Units,Tenure,Completion Date,Purchaser Address Indicator,Postal Code,Postal District,Postal Sector,Planning Region,Planning Area,Floor Number,BLK_NO,ROAD_NAME,BUILDING,POSTAL,LATITUDE,LONGITUDE,address_trunc,PPI,geometry,Police Centre,Centre geometry,Average Cases Per Year,Number of Primary Schools,Nearest Primary School,stations_1km,num_stations_1km,train_lines_1km,num_lines_1km,nearest_station_distance
0,0,EUPHONY GARDENS,"600,000","1,022.58",587,2010-01-01,5 JALAN MATA AYER #01-25,Resale,Strata,95,"6,316",-,Condominium,1,99 yrs from 25/02/1998,2001,HDB,759151,27,75,North Region,Mandai,1.0,5,JALAN MATA AYER,EUPHONY GARDENS,759151,1.424621,103.824899,5 JALAN MATA AYER,124.3,POINT (103.82490 1.42462),Yishun South NPC,POINT (103.83491 1.41532),33,7,705.752731,[],0,[],0,1207.822015
1,1,CARIBBEAN AT KEPPEL BAY,"1,909,000","1,485.43","1,285",2010-01-02,42 KEPPEL BAY DRIVE #01-111,Resale,Strata,138,"13,833",-,Condominium,1,99 yrs from 16/08/1999,2004,Private,098656,4,9,Central Region,Bukit Merah,1.0,42,KEPPEL BAY DRIVE,CARIBBEAN AT KEPPEL BAY,098656,1.266813,103.814674,42 KEPPEL BAY DRIVE,124.3,POINT (103.81467 1.26681),Bukit Merah West NPC,POINT (103.82340 1.28561),58,3,1233.947139,[HARBOURFRONT MRT STATION],1,[PURPLE],1,768.529003
2,2,ASTOR GREEN,"890,000","1,065.64",835,2010-01-03,7 PINE GROVE #10-01,Resale,Strata,99,"8,990",-,Condominium,1,99 yrs from 18/11/1991,1995,Private,597592,21,59,Central Region,Bukit Timah,10.0,7,PINE GROVE,ASTOR GREEN,597592,1.318459,103.776638,7 PINE GROVE,124.3,POINT (103.77664 1.31846),Clementi NPC,POINT (103.76656 1.31717),50,4,1039.586179,[DOVER MRT STATION],1,[GREEN],1,816.818037
3,3,ORCHID PARK CONDOMINIUM,"700,000","1,194.8",586,2010-01-03,81 YISHUN STREET 81 #06-01,Resale,Strata,111,"6,306",-,Condominium,1,99 yrs from 01/09/1991,1994,Private,768445,27,76,North Region,Yishun,6.0,81,YISHUN STREET 81,ORCHID PARK CONDOMINIUM,768445,1.413458,103.835186,81 YISHUN STREET 81,124.3,POINT (103.83519 1.41346),Yishun South NPC,POINT (103.83491 1.41532),33,5,509.516515,[KHATIB MRT STATION],1,[RED],1,501.364218
4,4,CARIBBEAN AT KEPPEL BAY,"1,700,000","1,313.21","1,295",2010-01-04,12 KEPPEL BAY DRIVE #10-13,Resale,Strata,122,"13,934",-,Condominium,1,99 yrs from 16/08/1999,2004,Private,098641,4,9,Central Region,Bukit Merah,10.0,12,KEPPEL BAY DRIVE,CARIBBEAN AT KEPPEL BAY,098641,1.266570,103.816587,12 KEPPEL BAY DRIVE,124.3,POINT (103.81659 1.26657),Bukit Merah West NPC,POINT (103.82340 1.28561),58,3,1253.733260,[HARBOURFRONT MRT STATION],1,[PURPLE],1,554.491114


In [ ]:
# Save train station gdf for interface purpose
train_station_gdf = train_station_gdf.to_crs(crs={"init":"EPSG:4326"})
train_station_gdf.to_csv('train_gdf.csv', header=True, index = False)

In [ ]:
for line in hdb_geocoded_gdf['train_lines_1km'].values: 
    if 'BROWN' in line: 
        print(line)

['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN']
['RED', 'BROWN

### 6. Remaining Lease and Age of Listing at the Point of Sale
Find remaining lease of unit

In [ ]:
def find_remaining_lease(df):
    '''Find the remaining lease period at the point of sale'''
    
    data = df.copy()
    data['Remaining Lease'] = np.nan
    
    for index, row in data.iterrows():
        curr_value = str(row['Tenure'])
        
        if (pd.isnull(curr_value)) or (curr_value == 'nan') : # No tenure 
            data.loc[index,'Tenure'] = 'Freehold'
            data.loc[index,'Remaining Lease'] = 999999
            
        elif curr_value == 'Freehold' : # Assigning 999999 years for Freehold units
            data.loc[index,'Remaining Lease'] = 999999
            
        else:
            curr_value = curr_value.split(' ')
            num_years = int(curr_value[0])
            start_date = datetime.datetime.strptime(curr_value[-1], '%d/%m/%Y')
            data.loc[index,'Remaining Lease'] = num_years - (data.loc[index,'Sale Date'].year - start_date.year)
            
    return data

hdb_geocoded_gdf = find_remaining_lease(hdb_geocoded_gdf)

Find the age of unit when it was sold

In [ ]:
def find_age_sold(df):
    '''Find the age of unit at the point of sale'''
    
    data = df.copy()
    data['Age Sold'] = np.nan
    
    for index, row in data.iterrows():
        completion_year =row['Completion Date'] 
        sale_year = row['Sale Date'].year
        
        # Take the starting year of the lease as the completion date for the listing
        if (completion_year == '-') & (row['Tenure'] != 'Freehold'):
            lease = row['Tenure']
            lease_start_date = pd.to_datetime(lease.split(' ')[-1], format = '%d/%m/%Y')
            completion_year = lease_start_date.year 
            age_sold = sale_year - completion_year
            data.loc[index, 'Age Sold'] = age_sold
        
        # Freehold units with no completion date
        elif (completion_year == '-') & (row['Tenure'] == 'Freehold'):# freehold, completion date is '-'
            continue
            
        else:
            completion_year = int(completion_year)
            age_sold = sale_year - completion_year
            data.loc[index, 'Age Sold'] = age_sold

    return data

hdb_geocoded_gdf = find_age_sold(hdb_geocoded_gdf)
hdb_geocoded_gdf.shape

(54764, 44)

### 7. One Hot Encoding

In [ ]:
def one_hot_encoding(df):
    ''' 
    One hot encoding on categorical variables
    1. Type of Area
    2. Planning Area
    3. Planning Region
    '''
    data = df.copy()

    data['Type of Area'] = data['Type of Area'].map({'Strata': 1, 'Land': 0})
    
    planning_area_dummies = pd.get_dummies(data['Planning Area'])
    data_combined = pd.concat([data, planning_area_dummies], axis=1)       
    
    return data_combined

hdb_geocoded_gdf = one_hot_encoding(hdb_geocoded_gdf)
hdb_geocoded_gdf.head()

,index,Project Name,Transacted Price ($),Area (SQFT),Unit Price ($ PSF),Sale Date,Address,Type of Sale,Type of Area,Area (SQM),Unit Price ($ PSM),Nett Price($),Property Type,Number of Units,Tenure,Completion Date,Purchaser Address Indicator,Postal Code,Postal District,Postal Sector,Planning Region,Planning Area,Floor Number,BLK_NO,ROAD_NAME,BUILDING,POSTAL,LATITUDE,LONGITUDE,address_trunc,PPI,geometry,Police Centre,Centre geometry,Average Cases Per Year,Number of Primary Schools,Nearest Primary School,stations_1km,num_stations_1km,train_lines_1km,num_lines_1km,nearest_station_distance,Remaining Lease,Age Sold,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Woodlands,Yishun
0,0,EUPHONY GARDENS,"600,000","1,022.58",587,2010-01-01,5 JALAN MATA AYER #01-25,Resale,1,95,"6,316",-,Condominium,1,99 yrs from 25/02/1998,2001,HDB,759151,27,75,North Region,Mandai,1.0,5,JALAN MATA AYER,EUPHONY GARDENS,759151,1.424621,103.824899,5 JALAN MATA AYER,124.3,POINT (103.82490 1.42462),Yishun South NPC,POINT (103.83491 1.41532),33,7,705.752731,[],0,[],0,1207.822015,87.0,9.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,CARIBBEAN AT KEPPEL BAY,"1,909,000","1,485.43","1,285",2010-01-02,42 KEPPEL BAY DRIVE #01-111,Resale,1,138,"13,833",-,Condominium,1,99 yrs from 16/08/1999,2004,Private,098656,4,9,Central Region,Bukit Merah,1.0,42,KEPPEL BAY DRIVE,CARIBBEAN AT KEPPEL BAY,098656,1.266813,103.814674,42 KEPPEL BAY DRIVE,124.3,POINT (103.81467 1.26681),Bukit Merah West NPC,POINT (103.82340 1.28561),58,3,1233.947139,[HARBOURFRONT MRT STATION],1,[PURPLE],1,768.529003,88.0,6.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,ASTOR GREEN,"890,000","1,065.64",835,2010-01-03,7 PINE GROVE #10-01,Resale,1,99,"8,990",-,Condominium,1,99 yrs from 18/11/1991,1995,Private,597592,21,59,Central Region,Bukit Timah,10.0,7,PINE GROVE,ASTOR GREEN,597592,1.318459,103.776638,7 PINE GROVE,124.3,POINT (103.77664 1.31846),Clementi NPC,POINT (103.76656 1.31717),50,4,1039.586179,[DOVER MRT STATION],1,[GREEN],1,816.818037,80.0,15.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,ORCHID PARK CONDOMINIUM,"700,000","1,194.8",586,2010-01-03,81 YISHUN STREET 81 #06-01,Resale,1,111,"6,306",-,Condominium,1,99 yrs from 01/09/1991,1994,Private,768445,27,76,North Region,Yishun,6.0,81,YISHUN STREET 81,ORCHID PARK CONDOMINIUM,768445,1.413458,103.835186,81 YISHUN STREET 81,124.3,POINT (103.83519 1.41346),Yishun South NPC,POINT (103.83491 1.41532),33,5,509.516515,[KHATIB MRT STATION],1,[RED],1,501.364218,80.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,4,CARIBBEAN AT KEPPEL BAY,"1,700,000","1,313.21","1,295",2010-01-04,12 KEPPEL BAY DRIVE #10-13,Resale,1,122,"13,934",-,Condominium,1,99 yrs from 16/08/1999,2004,Private,098641,4,9,Central Region,Bukit Merah,10.0,12,KEPPEL BAY DRIVE,CARIBBEAN AT KEPPEL BAY,098641,1.266570,103.816587,12 KEPPEL BAY DRIVE,124.3,POINT (103.81659 1.26657),Bukit Merah West NPC,POINT (103.82340 1.28561),58,3,1253.733260,[HARBOURFRONT MRT STATION],1,[PURPLE],1,554.491114,88.0,6.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# One hot encoding train_lines_1km column
mlb = MultiLabelBinarizer()
hdb_geocoded_gdf = hdb_geocoded_gdf.join(pd.DataFrame(mlb.fit_transform(hdb_geocoded_gdf.pop('train_lines_1km')),
                          columns=mlb.classes_,
                          index=hdb_geocoded_gdf.index))

In [ ]:
hdb_geocoded_gdf.head()

,index,Project Name,Transacted Price ($),Area (SQFT),Unit Price ($ PSF),Sale Date,Address,Type of Sale,Type of Area,Area (SQM),Unit Price ($ PSM),Nett Price($),Property Type,Number of Units,Tenure,Completion Date,Purchaser Address Indicator,Postal Code,Postal District,Postal Sector,Planning Region,Planning Area,Floor Number,BLK_NO,ROAD_NAME,BUILDING,POSTAL,LATITUDE,LONGITUDE,address_trunc,PPI,geometry,Police Centre,Centre geometry,Average Cases Per Year,Number of Primary Schools,Nearest Primary School,stations_1km,num_stations_1km,num_lines_1km,nearest_station_distance,Remaining Lease,Age Sold,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Woodlands,Yishun,BLUE,BROWN,GREEN,LRT,PURPLE,RED,YELLOW
0,0,EUPHONY GARDENS,"600,000","1,022.58",587,2010-01-01,5 JALAN MATA AYER #01-25,Resale,1,95,"6,316",-,Condominium,1,99 yrs from 25/02/1998,2001,HDB,759151,27,75,North Region,Mandai,1.0,5,JALAN MATA AYER,EUPHONY GARDENS,759151,1.424621,103.824899,5 JALAN MATA AYER,124.3,POINT (103.82490 1.42462),Yishun South NPC,POINT (103.83491 1.41532),33,7,705.752731,[],0,0,1207.822015,87.0,9.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,CARIBBEAN AT KEPPEL BAY,"1,909,000","1,485.43","1,285",2010-01-02,42 KEPPEL BAY DRIVE #01-111,Resale,1,138,"13,833",-,Condominium,1,99 yrs from 16/08/1999,2004,Private,098656,4,9,Central Region,Bukit Merah,1.0,42,KEPPEL BAY DRIVE,CARIBBEAN AT KEPPEL BAY,098656,1.266813,103.814674,42 KEPPEL BAY DRIVE,124.3,POINT (103.81467 1.26681),Bukit Merah West NPC,POINT (103.82340 1.28561),58,3,1233.947139,[HARBOURFRONT MRT STATION],1,1,768.529003,88.0,6.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,2,ASTOR GREEN,"890,000","1,065.64",835,2010-01-03,7 PINE GROVE #10-01,Resale,1,99,"8,990",-,Condominium,1,99 yrs from 18/11/1991,1995,Private,597592,21,59,Central Region,Bukit Timah,10.0,7,PINE GROVE,ASTOR GREEN,597592,1.318459,103.776638,7 PINE GROVE,124.3,POINT (103.77664 1.31846),Clementi NPC,POINT (103.76656 1.31717),50,4,1039.586179,[DOVER MRT STATION],1,1,816.818037,80.0,15.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,3,ORCHID PARK CONDOMINIUM,"700,000","1,194.8",586,2010-01-03,81 YISHUN STREET 81 #06-01,Resale,1,111,"6,306",-,Condominium,1,99 yrs from 01/09/1991,1994,Private,768445,27,76,North Region,Yishun,6.0,81,YISHUN STREET 81,ORCHID PARK CONDOMINIUM,768445,1.413458,103.835186,81 YISHUN STREET 81,124.3,POINT (103.83519 1.41346),Yishun South NPC,POINT (103.83491 1.41532),33,5,509.516515,[KHATIB MRT STATION],1,1,501.364218,80.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
4,4,CARIBBEAN AT KEPPEL BAY,"1,700,000","1,313.21","1,295",2010-01-04,12 KEPPEL BAY DRIVE #10-13,Resale,1,122,"13,934",-,Condominium,1,99 yrs from 16/08/1999,2004,Private,098641,4,9,Central Region,Bukit Merah,10.0,12,KEPPEL BAY DRIVE,CARIBBEAN AT KEPPEL BAY,098641,1.266570,103.816587,12 KEPPEL BAY DRIVE,124.3,POINT (103.81659 1.26657),Bukit Merah West NPC,POINT (103.82340 1.28561),58,3,1253.733260,[HARBOURFRONT MRT STATION],1,1,554.491114,88.0,6.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


## Final Data Cleaning

### Clean ENBLOC, Number of Units, and Price features

In [ ]:
def clean_enbloc(data):
    # Filtering out ENBLOC sales
    data = data[~data['Address'].str.contains('ENBLOC')]
    
    return data

def clean_number_of_units(data):
    # Filtering out transactions with more than 6 units sold
    data = data[data['Number of Units'] <= 6]
    
    return data

def clean_convert_price(data):
    data['Area (SQFT)'] = data['Area (SQFT)'].apply(lambda x: float(re.sub(',', '', str(x))))
    data['Area (SQM)'] = data['Area (SQM)'].apply(lambda x: float(re.sub(',', '', str(x))))
    data['Unit Price ($ PSM)'] = data['Unit Price ($ PSM)'].apply(lambda x: float(re.sub(',', '', x)))
    
    return data

In [ ]:
hdb_geocoded_gdf = clean_enbloc(hdb_geocoded_gdf)
hdb_geocoded_gdf = clean_number_of_units(hdb_geocoded_gdf)
hdb_geocoded_gdf = clean_convert_price(hdb_geocoded_gdf)

print(hdb_geocoded_gdf.shape)
hdb_geocoded_gdf.head()

(54764, 87)


,index,Project Name,Transacted Price ($),Area (SQFT),Unit Price ($ PSF),Sale Date,Address,Type of Sale,Type of Area,Area (SQM),Unit Price ($ PSM),Nett Price($),Property Type,Number of Units,Tenure,Completion Date,Purchaser Address Indicator,Postal Code,Postal District,Postal Sector,Planning Region,Planning Area,Floor Number,BLK_NO,ROAD_NAME,BUILDING,POSTAL,LATITUDE,LONGITUDE,address_trunc,PPI,geometry,Police Centre,Centre geometry,Average Cases Per Year,Number of Primary Schools,Nearest Primary School,stations_1km,num_stations_1km,num_lines_1km,nearest_station_distance,Remaining Lease,Age Sold,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Woodlands,Yishun,BLUE,BROWN,GREEN,LRT,PURPLE,RED,YELLOW
0,0,EUPHONY GARDENS,"600,000",1022.58,587,2010-01-01,5 JALAN MATA AYER #01-25,Resale,1,95.0,6316.0,-,Condominium,1,99 yrs from 25/02/1998,2001,HDB,759151,27,75,North Region,Mandai,1.0,5,JALAN MATA AYER,EUPHONY GARDENS,759151,1.424621,103.824899,5 JALAN MATA AYER,124.3,POINT (103.82490 1.42462),Yishun South NPC,POINT (103.83491 1.41532),33,7,705.752731,[],0,0,1207.822015,87.0,9.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,CARIBBEAN AT KEPPEL BAY,"1,909,000",1485.43,"1,285",2010-01-02,42 KEPPEL BAY DRIVE #01-111,Resale,1,138.0,13833.0,-,Condominium,1,99 yrs from 16/08/1999,2004,Private,098656,4,9,Central Region,Bukit Merah,1.0,42,KEPPEL BAY DRIVE,CARIBBEAN AT KEPPEL BAY,098656,1.266813,103.814674,42 KEPPEL BAY DRIVE,124.3,POINT (103.81467 1.26681),Bukit Merah West NPC,POINT (103.82340 1.28561),58,3,1233.947139,[HARBOURFRONT MRT STATION],1,1,768.529003,88.0,6.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,2,ASTOR GREEN,"890,000",1065.64,835,2010-01-03,7 PINE GROVE #10-01,Resale,1,99.0,8990.0,-,Condominium,1,99 yrs from 18/11/1991,1995,Private,597592,21,59,Central Region,Bukit Timah,10.0,7,PINE GROVE,ASTOR GREEN,597592,1.318459,103.776638,7 PINE GROVE,124.3,POINT (103.77664 1.31846),Clementi NPC,POINT (103.76656 1.31717),50,4,1039.586179,[DOVER MRT STATION],1,1,816.818037,80.0,15.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,3,ORCHID PARK CONDOMINIUM,"700,000",1194.80,586,2010-01-03,81 YISHUN STREET 81 #06-01,Resale,1,111.0,6306.0,-,Condominium,1,99 yrs from 01/09/1991,1994,Private,768445,27,76,North Region,Yishun,6.0,81,YISHUN STREET 81,ORCHID PARK CONDOMINIUM,768445,1.413458,103.835186,81 YISHUN STREET 81,124.3,POINT (103.83519 1.41346),Yishun South NPC,POINT (103.83491 1.41532),33,5,509.516515,[KHATIB MRT STATION],1,1,501.364218,80.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
4,4,CARIBBEAN AT KEPPEL BAY,"1,700,000",1313.21,"1,295",2010-01-04,12 KEPPEL BAY DRIVE #10-13,Resale,1,122.0,13934.0,-,Condominium,1,99 yrs from 16/08/1999,2004,Private,098641,4,9,Central Region,Bukit Merah,10.0,12,KEPPEL BAY DRIVE,CARIBBEAN AT KEPPEL BAY,098641,1.266570,103.816587,12 KEPPEL BAY DRIVE,124.3,POINT (103.81659 1.26657),Bukit Merah West NPC,POINT (103.82340 1.28561),58,3,1253.733260,[HARBOURFRONT MRT STATION],1,1,554.491114,88.0,6.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


### Outlier detection and removal (Discuss)

In [ ]:
# def get_outlier(data, bound='all'):
#     p_25 = data['Unit Price ($ PSM)'].describe()['25%']
#     p_75 = data['Unit Price ($ PSM)'].describe()['75%']
#     iqr = p_75 - p_25
    
#     lower_bound = max(0, p_25 - 1.5*iqr)
#     upper_bound = p_75 + 1.5*iqr
    
#     if bound=='all':
#         return data[(data['Unit Price ($ PSM)'] < lower_bound) | (data['Unit Price ($ PSM)'] > upper_bound)]
    
#     elif bound=='upper':
#         return data[data['Unit Price ($ PSM)'] > upper_bound]
    
#     elif bound=='lower':
#         return data[data['Unit Price ($ PSM)'] < lower_bound]

# df_outliers = get_outlier(hdb_geocoded_gdf, bound='all')
# df_outliers

### Missing Completion Date

In [ ]:
def find_completion_date(df):
    '''
    For units age sold = nan, find the completion date by searching postal code in full dataframe
    '''
    
    data = df.copy()
    # get the postal codes of units with missing completion date
    missing_completion_date = list(data[data['Age Sold'].isna()]['Postal Code'].unique())
    
    # look up completion using postal code on the full data set
    existing_completion_date = data[(data['Postal Code'].isin(missing_completion_date)) & (data['Completion Date'] != '-')][['Postal Code','Completion Date']]\
    .set_index('Postal Code').to_dict()['Completion Date']
    
    # assign completion date
    for index, row in data.iterrows():
        if (pd.isna(row['Age Sold'])) & (row['Postal Code'] in existing_completion_date):
            completion_year = existing_completion_date[row['Postal Code']]
            data.loc[index, 'Completion Date'] = completion_year
            
            sale_year = pd.to_datetime(row['Sale Date']).year
            age_sold = sale_year - int(completion_year)
            data.loc[index, 'Age Sold'] = age_sold
    
    return data

def manual_fill_completion_date(df, data_completion_date):
    '''
    Manually fill completion date for units with nan age sold. 
    '''
    
    data = df.copy()
    # get unique postal codes in manual fill df
    postal_code = list(data_completion_date['Postal Code'].unique())
    
    # assign completion date
    for index, row in data.iterrows():
        if (pd.isna(row['Age Sold'])) & (int(row['Postal Code']) in postal_code):
            completion_year = data_completion_date[data_completion_date['Postal Code'] == int(row['Postal Code'])].iloc[0]['Year']
            data.loc[index, 'Completion Date'] = str(completion_year)
            
            sale_year = pd.to_datetime(row['Sale Date']).year
            age_sold = sale_year - completion_year
            data.loc[index, 'Age Sold'] = age_sold
    
    return data

In [ ]:
hdb_geocoded_gdf = find_completion_date(hdb_geocoded_gdf)
hdb_geocoded_gdf = manual_fill_completion_date(hdb_geocoded_gdf, completion_date_manual)
hdb_geocoded_gdf.shape

(54764, 87)

### Final Cleaning

In [ ]:
# Load and Clean
hdb_geocoded_gdf.dropna(how='any', 
                        subset=['Age Sold'],
                        inplace=True)

hdb_geocoded_gdf = hdb_geocoded_gdf[hdb_geocoded_gdf['Type of Area']==1]
hdb_geocoded_gdf = hdb_geocoded_gdf[hdb_geocoded_gdf['Number of Units']==1]

# Remove inconsistent postal codes
hdb_geocoded_gdf = hdb_geocoded_gdf[hdb_geocoded_gdf['Postal Code']==hdb_geocoded_gdf['POSTAL']]

Create preliminary dataset

In [ ]:
preliminary_df = hdb_geocoded_gdf.copy()

In [ ]:
preliminary_df = preliminary_df.drop('index', axis=1)
preliminary_df.to_csv('preliminary_dataset.csv', index=False)

In [ ]:
# KIV Postal Code, Planning Region, Planning Area, BLK_NO, ROAD_NAME, BUILDING, Police Centre
# Generate distance for nearest school (pre+k, pri)
# OHE train line colours
preliminary_df.drop(columns=['Project Name', 
                               'Area (SQFT)',
                               'Unit Price ($ PSF)',
                               # 'Address',
                               'Type of Sale',
                               'Type of Area',
                               'Nett Price($)',
                               'Number of Units',
                               'Tenure',
                               'Completion Date',
                               'Purchaser Address Indicator',
                               'Postal District',
                               'Postal Sector',
                               'LATITUDE',
                               'LONGITUDE',
                               'address_trunc',
                               'geometry',
                               'Centre geometry',
                               'stations_1km',
                               'num_lines_1km'], inplace=True)

In [ ]:
# KIV Postal Code, Planning Region, Planning Area, BLK_NO, ROAD_NAME, BUILDING, Police Centre
# Generate distance for nearest school (pre+k, pri)
# OHE train line colours
hdb_geocoded_gdf.drop(columns=['Project Name', 
                               'Area (SQFT)',
                               'Unit Price ($ PSF)',
                               'Address',
                               'Type of Sale',
                               'Type of Area',
                               'Nett Price($)',
                               'Number of Units',
                               'Tenure',
                               'Completion Date',
                               'Purchaser Address Indicator',
                               'Postal District',
                               'Postal Sector',
                               'LATITUDE',
                               'LONGITUDE',
                               'address_trunc',
                               'geometry',
                               'Centre geometry',
                               'stations_1km',
                               'num_lines_1km'], inplace=True)

In [ ]:
print(hdb_geocoded_gdf.shape)
hdb_geocoded_gdf

(54754, 67)


,index,Transacted Price ($),Sale Date,Area (SQM),Unit Price ($ PSM),Property Type,Postal Code,Planning Region,Planning Area,Floor Number,BLK_NO,ROAD_NAME,BUILDING,POSTAL,PPI,Police Centre,Average Cases Per Year,Number of Primary Schools,Nearest Primary School,num_stations_1km,nearest_station_distance,Remaining Lease,Age Sold,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Woodlands,Yishun,BLUE,BROWN,GREEN,LRT,PURPLE,RED,YELLOW
0,0,"600,000",2010-01-01,95.0,6316.0,Condominium,759151,North Region,Mandai,1.0,5,JALAN MATA AYER,EUPHONY GARDENS,759151,124.3,Yishun South NPC,33,7,705.752731,0,1207.822015,87.0,9.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,"1,909,000",2010-01-02,138.0,13833.0,Condominium,098656,Central Region,Bukit Merah,1.0,42,KEPPEL BAY DRIVE,CARIBBEAN AT KEPPEL BAY,098656,124.3,Bukit Merah West NPC,58,3,1233.947139,1,768.529003,88.0,6.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,2,"890,000",2010-01-03,99.0,8990.0,Condominium,597592,Central Region,Bukit Timah,10.0,7,PINE GROVE,ASTOR GREEN,597592,124.3,Clementi NPC,50,4,1039.586179,1,816.818037,80.0,15.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,3,"700,000",2010-01-03,111.0,6306.0,Condominium,768445,North Region,Yishun,6.0,81,YISHUN STREET 81,ORCHID PARK CONDOMINIUM,768445,124.3,Yishun South NPC,33,5,509.516515,1,501.364218,80.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
4,4,"1,700,000",2010-01-04,122.0,13934.0,Condominium,098641,Central Region,Bukit Merah,10.0,12,KEPPEL BAY DRIVE,CARIBBEAN AT KEPPEL BAY,098641,124.3,Bukit Merah West NPC,58,3,1253.733260,1,554.491114,88.0,6.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54759,54759,"1,580,000",2020-12-31,192.0,8229.0,Condominium,798592,North East Region,Hougang,4.0,100,GERALD DRIVE,SELETAR SPRINGS CONDOMINIUM,798592,153.3,Hougang NPC,66,11,526.868822,3,464.021930,76.0,20.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
54760,54760,"870,000",2020-12-31,111.0,7838.0,Executive Condominium,545040,North East Region,Sengkang,14.0,2,RIVERVALE LINK,PARK GREEN,545040,153.3,Hougang NPC,66,15,535.286185,3,490.669867,80.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
54761,54761,"1,850,000",2020-12-31,142.0,13028.0,Condominium,436904,Central Region,Kallang,14.0,120,TANJONG RHU ROAD,CASUARINA COVE,436904,153.3,Geylang NPC,133,0,2075.609094,1,600.145285,72.0,24.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
54762,54762,"808,000",2020-12-31,58.0,13931.0,Condominium,677669,West Region,Bukit Panjang,6.0,7,DAIRY FARM HEIGHTS,THE SKYWOODS,677669,153.3,Bukit Batok NPC,51,3,397.651025,2,539.116552,91.0,4.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


## Saving Output

In [ ]:
hdb_geocoded_gdf = hdb_geocoded_gdf.drop('index', axis=1)

# Saving finalized dataframe into shared google drive
hdb_geocoded_gdf.to_csv('final_dataset.csv', index=False)
#!cp final_dataset.csv "drive/My Drive/Primary Datasets/"